In [4]:
import os
from dotenv import load_dotenv
import pyodbc
import pandas as pd

# Charger les variables d'environnement
load_dotenv()

server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:

"carateris.csv"
"dimen.csv"
"produit.csv"

In [5]:
import pandas as pd
import pyodbc
from datetime import datetime

# Lecture des fichiers CSV avec spécification des types
df_produit = pd.read_csv('produit.csv', dtype={
    'ID_Produit': int,
    'URL_Produit': str,
    'Prix': float,  # Changé en float au cas où il y aurait des décimales
    'Info_generale': str,
    'Descriptif': str,
    'Note': str,
    'Date_scrap': str,
    'Marque': str
})

df_caracteristiques = pd.read_csv('carateris.csv', dtype={
    'ID_Caracteristique': int,
    'Consommation': str,
    'Indice_Pluie': str,
    'Bruit': int,
    'Saisonalite': str,
    'Type_Vehicule': str,
    'Runflat': str,
    'ID_Produit': int
})

df_dimensions = pd.read_csv('dimen.csv', dtype={
    'ID_Dimension': int,
    'Largeur': int,
    'Hauteur': int,
    'Diametre': int,
    'Charge': int,
    'Vitesse': str,
    'ID_Produit': int
})

# Fonction pour gérer IDENTITY_INSERT
def disable_identity_insert(cursor, table_name, disable=True):
    if disable:
        cursor.execute(f"SET IDENTITY_INSERT {table_name} OFF")
    else:
        cursor.execute(f"SET IDENTITY_INSERT {table_name} ON")

try:
    # Insertion des produits
    disable_identity_insert(cursor, 'Produit', False)
    
    for _, row in df_produit.iterrows():
        # Conversion de la note en string vide si c'est 'note inconnue'
        note = row['Note'] if row['Note'] != 'note inconnue' else ''
        
        cursor.execute("""
            INSERT INTO Produit (ID_Produit, URL_Produit, Prix, Info_generale, 
                               Descriptif, Note, Date_scrap, Marque)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['ID_Produit']),
            str(row['URL_Produit']),
            float(row['Prix']),
            str(row['Info_generale']),
            str(row['Descriptif']),
            str(note),
            datetime.strptime(row['Date_scrap'], '%Y-%m-%d').date(),
            str(row['Marque'])
        ))
    
    disable_identity_insert(cursor, 'Produit', True)
    
    # Insertion des caractéristiques
    disable_identity_insert(cursor, 'Caracteristiques', False)
    
    for _, row in df_caracteristiques.iterrows():
        cursor.execute("""
            INSERT INTO Caracteristiques (ID_Caracteristique, Consommation, 
                                        Indice_Pluie, Bruit, Saisonalite, 
                                        Type_Vehicule, Runflat, ID_Produit)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['ID_Caracteristique']),
            str(row['Consommation']),
            str(row['Indice_Pluie']),
            int(row['Bruit']),
            str(row['Saisonalite']),
            str(row['Type_Vehicule']),
            str(row['Runflat']),
            int(row['ID_Produit'])
        ))
    
    disable_identity_insert(cursor, 'Caracteristiques', True)
    
    # Insertion des dimensions
    disable_identity_insert(cursor, 'Dimensions', False)
    
    for _, row in df_dimensions.iterrows():
        cursor.execute("""
            INSERT INTO Dimensions (ID_Dimension, Largeur, Hauteur, Diametre, 
                                  Charge, Vitesse, ID_Produit)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            int(row['ID_Dimension']),
            int(row['Largeur']),
            int(row['Hauteur']),
            int(row['Diametre']),
            int(row['Charge']),
            str(row['Vitesse']),
            int(row['ID_Produit'])
        ))
    
    disable_identity_insert(cursor, 'Dimensions', True)
    
    # Validation des modifications
    cnxn.commit()
    print("Insertion des données réussie!")

except Exception as e:
    print(f"Une erreur est survenue: {str(e)}")
    cnxn.rollback()

finally:
    # Fermeture de la connexion
    cnxn.close()


Insertion des données réussie!
